In [ ]:
# please upload data directory to Google drive
# use Google Drive to load data
from google.colab import drive, files
import zipfile
drive.mount('/content/drive')

zipname = 'supports.zip'
uploaded = files.upload()
# Extract the zip file
with zipfile.ZipFile(zipname, 'r') as zip_ref:
  zip_ref.extractall()  # Extract all files to the current directory

In [1]:
!pip install -r "requirements.txt"
!pip install torchinfo
!pip install torcheval
import torch
from sklearn.model_selection import train_test_split
from models.AST import AST
import os
import librosa
import numpy as np
from torchinfo import summary
from torch import nn
from torch.optim import Adam
from Utils import create_dataloader, k_fold_cross_validation
from torcheval.metrics import MulticlassAUROC, MulticlassF1Score

# limit GPU usage (Can be ignored)

In [2]:
# torch.cuda.set_per_process_memory_fraction(0.625)

# Read Data 

In [3]:
root = '/content/drive/MyDrive/Data/genres_original'
genres = os.listdir(root)
x = []
y = []
length = []
sr = 16*1000
for genre in genres:
    genre_root = os.path.join(root, genre)
    audios = os.listdir(genre_root)
    for audio in audios:
        audio_path = os.path.join(genre_root, audio)
        signal, sr = librosa.load(audio_path, sr=sr)
        x.append(signal)
        length.append(len(signal))
        y.append(genres.index(genre))
min_length = min(length)
print("finsh reading data")

finsh reading data


# Conversion, Compression and Normalisation

In [4]:
top_db = 80
for i in range(len(x)):
    signal = x[i][:min_length]
    mel_spect = librosa.feature.melspectrogram(y=signal,sr=sr,n_fft=1024) # convert signals to mel spectrogram
    mel_spect = librosa.power_to_db(mel_spect, ref=np.max, top_db=top_db) # log compression
    x[i] = mel_spect/-top_db # normalisation
print("finish conversion and compression")

finish conversion and compression


# Split Data

In [5]:
x = np.asarray(x)
x = x.transpose((0,2,1))
y = np.asarray(y)
print(x.shape)

(1000, 936, 128)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                    stratify=y,shuffle=True)
# k-fold cross validation
k = 5
xs_train, ys_train, xs_valid, ys_valid = k_fold_cross_validation(x_train,y_train,k)
print("finish splitting data")

finish splitting data


In [7]:
batch_size = 16
dataloaders_train = []
dataloaders_valid = []
for i in range(k):
    dataloaders_train.append(create_dataloader(xs_train[i], ys_train[i], batch_size=batch_size))
    dataloaders_valid.append(create_dataloader(xs_valid[i], ys_valid[i], batch_size=batch_size))
dataloader_test = create_dataloader(x_test, y_test, batch_size=batch_size)
print("finish creating dataloaders")

finish creating dataloaders


In [8]:
model = AST(10,input_fdim=x.shape[2],input_tdim=x.shape[1])
model.cuda()
loss_function = nn.CrossEntropyLoss()
opt = Adam(model.parameters(), lr=1e-5)
summary(model,[(batch_size,x.shape[1],x.shape[2])])

frequncey stride=10, time stride=10
number of patches=1116


E:\learning\master\1.2\COMP47700SA\Music-Genre-Classification\venv\lib\site-packages\timm\models\vision_transformer.py:91: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  x = F.scaled_dot_product_attention(


Layer (type:depth-idx)                   Output Shape              Param #
AST                                      [16, 10]                  2,398,160
├─VisionTransformerDistilled: 1-3        --                        (recursive)
│    └─ASTPatchEmbed: 2-1                [16, 1116, 768]           --
│    │    └─Conv2d: 3-1                  [16, 768, 12, 93]         197,376
│    └─Dropout: 2-2                      [16, 1118, 768]           --
├─ModuleList: 1-2                        --                        --
│    └─Block: 2-3                        [16, 1118, 768]           --
│    │    └─LayerNorm: 3-2               [16, 1118, 768]           1,536
│    │    └─Attention: 3-3               [16, 1118, 768]           2,362,368
│    │    └─Identity: 3-4                [16, 1118, 768]           --
│    │    └─Identity: 3-5                [16, 1118, 768]           --
│    │    └─LayerNorm: 3-6               [16, 1118, 768]           1,536
│    │    └─Mlp: 3-7                     [16, 1118,

In [9]:
for i in range(15):
    print("-------epoch  {} -------".format(i + 1))
    epoch_loss = 0
    epoch_accuracy = 0
    for j in range(k):
        print(f'fold {j+1}:')
        loss_train = 0
        accuracy_train = 0
        train_size = 0
        for batch_idx, (data, target) in enumerate(dataloaders_train[j]):
            model.train()
            output = model(data)
            loss = loss_function(output, target)
            opt.zero_grad()
            loss.backward()
            opt.step()
            loss_train += loss.item()*len(data)
            accuracy = (output.argmax(1) == target).sum()
            accuracy_train += accuracy
            train_size += len(data)
        print("train set loss: {}".format(loss_train/train_size))
        print("train set accuracy: {}".format(accuracy_train /train_size))

        loss_valid = 0
        accuracy_valid = 0
        valid_size = 0
        for batch_idx, (data, target) in enumerate(dataloaders_valid[j]):
            model.eval()
            with torch.no_grad():
                output = model(data)
                loss = loss_function(output, target)
                loss_valid += loss.item()*len(data)
                accuracy = (output.argmax(1) == target).sum()
                accuracy_valid += accuracy
                valid_size += len(data)
        print("valid set loss: {}".format(loss_valid/valid_size))
        print("valid set accuracy: {}".format(accuracy_valid/valid_size))
        epoch_loss += loss_valid/valid_size
        epoch_accuracy += accuracy_valid/valid_size
    print("epoch loss: {}".format(epoch_loss/k))
    print("epoch accuracy: {}".format(epoch_accuracy/k))
print("finish training")

-------epoch  1 -------
fold 1:
train set loss: 2.3001953125
train set accuracy: 0.10625000298023224
valid set loss: 2.2759765625
valid set accuracy: 0.10625000298023224
fold 2:
train set loss: 2.28955078125
train set accuracy: 0.11874999850988388
valid set loss: 2.2083984375
valid set accuracy: 0.21250000596046448
fold 3:
train set loss: 2.223583984375
train set accuracy: 0.19375000894069672
valid set loss: 2.14140625
valid set accuracy: 0.25
fold 4:
train set loss: 2.1794921875
train set accuracy: 0.23281250894069672
valid set loss: 2.0880859375
valid set accuracy: 0.29375001788139343
fold 5:
train set loss: 2.145654296875
train set accuracy: 0.3140625059604645
valid set loss: 2.02900390625
valid set accuracy: 0.38750001788139343
epoch loss: 2.1485742187500003
epoch accuracy: 0.2500000298023224
-------epoch  2 -------
fold 1:
train set loss: 2.119677734375
train set accuracy: 0.33125001192092896
valid set loss: 2.01943359375
valid set accuracy: 0.4937500059604645
fold 2:
train set lo

In [10]:
loss_test = 0
accuracy_test = 0
AUC_test = 0
f1_score_test = 0
test_size = 0
for batch_idx, (data, target) in enumerate(dataloader_test):
    model.eval()
    with torch.no_grad():
        output = model(data)
        loss = loss_function(output, target)
        loss_test += loss.item()*len(data)
        accuracy = (output.argmax(1) == target).sum()
        accuracy_test += accuracy
        test_size += len(data)
        auc = MulticlassAUROC(num_classes=10)
        auc.update(output, target)
        AUC_test += auc.compute()*len(data)
        f1 = MulticlassF1Score(num_classes=10)
        f1.update(output,target)
        f1_score_test += f1.compute()*len(data)        
print("test set loss: {}".format(loss_test/test_size))
print("test set accuracy: {}".format(accuracy_test/test_size))
print("test set AUC: {}".format(AUC_test/test_size))
print("test set f1-score: {}".format(f1_score_test/test_size))

test set loss: 1.907265625
test set accuracy: 0.7599999904632568
test set AUC: 0.8417731523513794
test set f1-score: 0.7599999904632568
